In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # for measuring model
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

import random
from collections import Counter

In [3]:
# import data
df = pd.read_df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv", sep = ",")
df

In [11]:
# check for null values and data types
null_summary = pd.DataFrame({
    'column': df.columns,
    'dtype': df.dtypes.values,
    'null_count': df.isnull().sum().values
})
null_summary

,column,dtype,null_count
0,lead_source,object,0
1,industry,object,0
2,number_of_courses_viewed,int64,0
3,annual_income,float64,0
4,employment_status,object,0
5,location,object,0
6,interaction_count,int64,0
7,lead_score,float64,0
8,converted,int64,0


In [10]:
# Separate categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
numerical_cols = df.select_dtypes(include=['number', 'float64', 'int64']).columns

# replace nulls with 'NA' for cat and 0 for num
df[categorical_cols] = df[categorical_cols].fillna('NA')
df[numerical_cols] = df[numerical_cols].fillna(0.0)